## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x,a):
    return 2.*x/(x + float(a)) - 1.

def f_invtransform(s,a):
    return float(a)*(1. + s)/(1. - s)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample,img_size):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=img_size
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict
    

In [6]:
# kappa=4
# img_size=64

kappa=10
img_size=128

In [7]:
val_data_dict={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing',
              '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube'}

In [8]:
### Read validation data
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins,kappa)   ### scale to (-1,1)

bkgnd_dict={}
num_bkgnd=100
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=[0,1,2,3]
### Extract validation data   
for label in labels_lst:
    fname=val_data_dict[str(img_size)]+'/Om0.3_Sg%s_H70.0.npy'%(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:,:]
    samples=f_transform(samples,kappa)
    dict_val=f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
# del samples


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/Om0.3_Sg0.5_H70.0.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/Om0.3_Sg0.65_H70.0.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/Om0.3_Sg0.8_H70.0.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/Om0.3_Sg1.1_H70.0.npy


In [9]:
# print(kappa)

In [10]:
df_runs=pd.DataFrame([])

In [ ]:
### bkgnd: Get bkgnd chi-squares for different sigma wrt one standard sigma 
lst=[0.5,0.65,0.8,1.1]
param_label=0.65
for count,run in enumerate(lst):
    num=100
    fname=val_data_dict[str(img_size)]+'/Om0.3_Sg%s_H70.0.npy'%(sigma_lst[count])
    print(fname)    
    samples=np.load(fname,mmap_mode='r')[1000:1000+num][:,0,:,:,:]
    samples=f_transform(samples,kappa)

    dict1={'label':'ref_%s'%(run),'fname':''}
    dict1['num_imgs']=num
    dict1['sigma']=run
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(samples,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    
    df_runs=df_runs.append(dict1,ignore_index=True)


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/Om0.3_Sg0.5_H70.0.npy


In [ ]:
df_runs

### Add pytorch runs

In [ ]:
## Pytorch code
### Load images

parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'


# lst=[
# #         ['20210628_73858_cgan_128_nodes1_lr0.0002_fm0_goodrun/',24970],
# #      ['20210703_50233_cgan_128_nodes1_lr0.0002_vary_fm0_goodrun/',19360],
# #      ['20210723_72050_cgan_128_nodes1_lr0.00002_finetune/',5070],
#      ['20210723_74424_cgan_128_nodes1_lr0.000002_finetune/',1950],
# #      ['20210726_173009_cgan_128_nodes1_lr0.000002_finetune/',3010],
#     ]

lst=[['20230208_111749_cgan_128_lr0.0002_fm00_sp0.1_kappa10_gpus3_newspecloss_no_var_good_run/',43810],
     ['20230208_111749_cgan_128_lr0.0002_fm00_sp0.1_kappa10_gpus3_newspecloss_no_var_good_run/',46440],
     ['20230208_111749_cgan_128_lr0.0002_fm00_sp0.1_kappa10_gpus3_newspecloss_no_var_good_run/',46590],
     ['20230208_111749_cgan_128_lr0.0002_fm00_sp0.1_kappa10_gpus3_newspecloss_no_var_good_run/',48830],
     ['20230208_111749_cgan_128_lr0.0002_fm00_sp0.1_kappa10_gpus3_newspecloss_no_var_good_run/',53170],
    ]
    
# Prediction images 

for param_label in sigma_lst:
    for count,run in enumerate(lst):
        fldr,step=run[0],run[1]
#         print(count,run)
        prefix='run_%s_%s_%s'%(count,step,param_label)
        # prefix='best_%s'%(param_label)
        key=prefix
        main_dir=parent_dir+fldr
        print(key)
        fname=glob.glob(main_dir+'images/'+'inference_*label-{0}_*_step-{1}.npy'.format(param_label,step))[0]
        dict1={'label':key,'fname':fname}
        images=np.load(dict1['fname'])     ### No transform needed here

        images=images[:,0,:,:,:]
        print(images.shape)
        print(np.max(images),np.min(images))

        dict1['num_imgs']=images.shape[0]
        dict1['sigma']=param_label
        print(dict1)
        ### Compute spectrum and histograms
        dict_sample=f_compute_hist_spect(images,bins)
        ### Compute chi squares
        dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)

        dict1.update(dict_sample)
        dict1.update(dict_chisqrs)
        del(images)
        df_runs=df_runs.append(dict1,ignore_index=True)

In [ ]:
df_runs.columns

In [ ]:
col_list=['label','sigma', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [ ]:
df_runs

### Compare chi-squares

In [ ]:
chi_keys=['label','sigma','chi_1','chi_spec1', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

## Plot

In [ ]:
def f_plot_hist_spec(df,ref_list,plot_type,img_size):

    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1,kappa)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y=row.spec_val
#             yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            yerr=row.spec_sdev
            x=np.arange(len(y))
            y=x**2*y; yerr=x**2*yerr ## Plot k^2 P(y)
            plt.fill_between(x, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x, y, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            print(row.sigma)
            dict_bkg=bkgnd_dict[str(row.sigma)]

            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)
    
            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size)
            plt.legend()
            plt.title("relative spectrum")
            
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:,0]
            print(images.shape)
            # f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
            images=np.load(row.fname)[:,0,:,:,:]
            images=f_invtransform(images,kappa)
            images=np.sum(images[:8],axis=3)
            images=f_transform(images,kappa)
            print(images.shape)
            f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
    ### Plot input data
    for sig in ref_list: 
        dict_bkg=bkgnd_dict[str(sig)]
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x,kappa)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)

        if plot_type=='spec':
    #         y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            x=np.arange(len(y))
            y=x**2*y; yerr=x**2*yerr ## Plot k^2 P(y)
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)


        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)    

        if plot_type in ['hist','spec']:     
            plt.yscale('log')
    if plot_type in ['hist','spec','spec_relative']: 
        plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
        plt.show()

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_runs[:-1],dict_val,'grid',128)


In [ ]:
def f_plot_grid(arr,cols=16,fig_size=(15,5)):
    ''' Plot a grid of images
    '''
    size=arr.shape[0]    
    rows=int(np.ceil(size/cols))
    print(rows,cols)
    
    fig,axarr=plt.subplots(rows,cols,figsize=fig_size, gridspec_kw = {'wspace':0, 'hspace':0})
    if rows==1: axarr=np.reshape(axarr,(rows,cols))
    if cols==1: axarr=np.reshape(axarr,(rows,cols))
    
    for i in range(min(rows*cols,size)):
        row,col=int(i/cols),i%cols
        try: 
            axarr[row,col].imshow(arr[i],origin='lower', cmap='Reds', extent = [0, 128, 0, 128], norm=Normalize(vmin=-1., vmax=1.))
        # Drop axis label
        except Exception as e:
            print('Exception:',e)
            pass
        temp=plt.setp([a.get_xticklabels() for a in axarr[:-1,:].flatten()], visible=False)
        temp=plt.setp([a.get_yticklabels() for a in axarr[:,1:].flatten()], visible=False)
        
    plt.show()

In [ ]:
df_runs.columns

## Widget

In [ ]:
df_runs

In [ ]:
def f_widget_compare(df,labels_list,bkgnd,ref_list,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,ref_list,plot_type,img_size)

interact_manual(f_widget_compare,df=fixed(df_runs),
                ref_list=SelectMultiple(options=sigma_lst),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                img_size=fixed(img_size),
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

In [22]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_imgs,chi_2,chi_spec2
0,ref_0.5,0.109239,1.656476,100.0,0.183163,320.783675
1,ref_0.65,0.000324,0.015009,100.0,0.000336,4.701211
2,ref_0.8,0.068254,0.569226,100.0,0.094834,162.202894
3,ref_1.1,0.494611,3.583120,100.0,0.530161,1016.374215
4,run_0_43810_0.5,0.063813,13.467691,128.0,0.116157,3023.717309
5,run_1_46440_0.5,0.161857,13.161061,128.0,0.287400,4378.784361
6,run_2_46590_0.5,0.035017,14.136370,128.0,0.063444,3026.780563
7,run_3_48830_0.5,0.060425,13.557889,128.0,0.107994,3871.191467
8,run_4_53170_0.5,0.083718,12.709012,128.0,0.154156,4377.184158
9,run_0_43810_0.65,0.038430,13.635725,128.0,0.063865,3235.395235


### Test 

In [36]:
df_runs

,label,sigma,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,ref_0.5,0.50,0.109239,0.098660,1.656476,53.655213,0.010228,0.000351,0.183163,0.813243,320.783675,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.006107039472769319, 0.001708507378728206, 0...","[2.1421856780648243, 0.9002978088140488, 0.037...","[15270326.767257554, 4351041.036634038, 203297...","[51932415.73760885, 29315042.295884848, 175193...",100.0,
1,ref_0.65,0.65,0.000324,0.000204,0.015009,48.408010,0.000037,0.000083,0.000336,0.998839,4.701211,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.007065483208818906, 0.00193903452548504, 0....","[2.5296693597435964, 0.7676551758885383, 0.041...","[18862742.83217792, 6536240.444012119, 3375343...","[66271626.77364769, 36146411.37348758, 2156111...",100.0,
2,ref_0.8,0.80,0.068254,0.050819,0.569226,53.107753,0.014400,0.003035,0.094834,1.072713,162.202894,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.007365556262997008, 0.0020303304654039575, ...","[2.839086867570879, 0.6644645259618762, 0.0416...","[19890159.01334602, 6966094.56347041, 3584763....","[75811871.34500581, 42736433.21354594, 2552092...",100.0,
3,ref_1.1,1.10,0.494611,0.279598,3.583120,59.763561,0.127270,0.087743,0.530161,1.343539,1016.374215,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.008596313141089117, 0.002317982288450415, 0...","[3.2378507573604605, 0.5324310088634489, 0.040...","[37148557.025807776, 11095966.403190607, 56482...","[107833301.53327806, 56429118.20946207, 330280...",100.0,
4,run_0_43810_0.5,0.50,0.063813,0.062567,13.467691,48.409729,0.001060,0.000186,0.116157,0.779128,3023.717309,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.004607276388324736, 0.0015026635286175239, ...","[2.4602400619536646, 0.7959885233826935, 0.039...","[11832024.957240315, 4126709.665918186, 211236...","[51752101.40962492, 32920478.245031025, 204033...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,run_1_46440_0.5,0.50,0.161857,0.152507,13.161061,51.559085,0.002488,0.006862,0.287400,0.622587,4378.784361,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.003414635059902128, 0.001005201985242921, 0...","[2.645920524559915, 0.7354866236262023, 0.0371...","[12410401.377843527, 4295242.614559247, 204370...","[64648488.99442187, 33128902.36642136, 1967500...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,run_2_46590_0.5,0.50,0.035017,0.034080,14.136370,51.096115,0.000415,0.000522,0.063444,0.706026,3026.780563,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.004017486069364319, 0.0012655977700038066, ...","[2.375733968289569, 0.8236994110699745, 0.0386...","[12988288.148690777, 3780599.3913741936, 19296...","[55034761.79709118, 32513803.725687683, 201093...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,run_3_48830_0.5,0.50,0.060425,0.057297,13.557889,51.209220,0.000641,0.002487,0.107994,1.194171,3871.191467,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.0072399351324084465, 0.0024490867202279134,...","[2.4489107910543697, 0.8005431147292255, 0.037...","[9936884.886715291, 2888729.193981506, 1821936...","[41556994.294199355, 25424609.912625197, 17572...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,run_4_53170_0.5,0.50,0.083718,0.082303,12.709012,52.746647,0.000375,0.001041,0.154156,1.321038,4377.184158,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.008292826345634734, 0.0025796725935612465, ...","[2.5090458812135865, 0.7785668636984229, 0.040...","[13747988.147267746, 4185289.308710147, 194360...","[59877691.90619843, 33884323.930785246, 204597...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,run_0_43810_0.65,0.65,0.038430,0.033521,13.635725,49.823303,0.004079,0.000829,0.063865,0.679771,3235.395235,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.004893630380788034, 0.0015340702993479202, ...","[2.787930222141549, 0.685623181063204, 0

In [47]:
a1=np.ones((5,5,5))
print(a1.shape)

(5, 5, 5)


In [49]:

np.sum(a1,axis=2).shape

(5, 5)